In [ ]:
import cv2
import os

In [ ]:
def run_tracker(video_path, tracker_type="CSRT", skip_seconds=2, num_frames=15):
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    if fps == 0: fps = 30
    frames_to_skip = int(fps * skip_seconds)

    for _ in range(frames_to_skip):
        video.read()

    success, frame = video.read()
    if not success or frame is None:
        return

    bbox = cv2.selectROI("Select Object", frame, False)
    cv2.destroyAllWindows()

    if tracker_type == "KCF":
        tracker = cv2.TrackerKCF.create()
    else:
        tracker = cv2.TrackerCSRT.create()

    tracker.init(frame, bbox)

    for i in range(num_frames):
        success, frame = video.read()
        if not success: break

        success_track, bbox = tracker.update(frame)
        if success_track:
            x, y, w, h = [int(v) for v in bbox]
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
            cv2.putText(frame, f"{tracker_type} Frame: {i}", (20, 40),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

        cv2.imwrite(f"tracking_results/{tracker_type}_frame_{i:02d}.jpg", frame)

    video.release()

In [ ]:
run_tracker("videoplayback.mp4", "CSRT", skip_seconds=2)